In [3]:
!pip install pyspark

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crear la sesión Spark
spark = SparkSession.builder.appName("keywordCount").getOrCreate()

# Leer el archivo CSV
csv_path = ".csv/general.csv"
csv_df = spark.read.csv(csv_path, header=True, inferSchema=True)

# Filtrar registros con columnas no nulas
filtered_df = csv_df.filter(col('abstract').isNotNull())

# MAP-REDUCE
# Seleccionar el campo abstract y convertirlo en un RDD
words_rdd = filtered_df.select("abstract").rdd.flatMap(lambda x: x[0].split())

# Lista de palabras
words_to_count = ["Science", "artificial", "intelligence", "The"]
words_to_count = [word.lower() for word in words_to_count]

# Contar las palabras
counts_rdd = words_rdd.filter(lambda word: word.lower() in words_to_count).map(lambda word: (word.lower(), 1)).reduceByKey(lambda a, b: a + b)

# Mostrar los resultados
print(counts_rdd.collect())

# Guardar los resultados en CSV
counts_df = counts_rdd.toDF(["word", "frequency"])  # convertir a DataFrame la lista resultado
counts_df.write.mode("overwrite").csv("./dynamicdata/Keywords_temp")

# Renombrar el archivo CSV resultante
spark.fs.rename("./dynamicdata/Keywords_temp/part-00000.csv", "./dynamicdata/Keywords.csv")

# Cerrar la sesión de Spark
spark.stop()


RuntimeError: Java gateway process exited before sending its port number